In [45]:
from dataclasses import dataclass
from math import gcd, ceil
import re
from collections import Counter, defaultdict, namedtuple, deque
from itertools import product, chain, combinations, combinations_with_replacement, permutations, accumulate, zip_longest
import numpy as np
from matplotlib import pyplot as plt
import networkx as nx

from aocutils.common import copy_func, patch_to, patch, to_int, ints, flatten, zippify, multidict, rev, data, quantify, atom, atoms, list_atoms, list_multiply, mapt
from aocutils.grid import gridneigh, arr_to_dict, grid_to_dict, neighbors, arr_neighbors, iterate, Dim, dimensions, positive, manhattan, conv1d, conv2d, rotate
from aocutils.maze import graphparse, bfs, dijkstra, distances, get_path, dfs
from aocutils.math import gcd, factors, lcm, crt, mul_inv, Segment, lis, angle, all_combinations, all_permutations, mst
from aocutils.special import md5, binarysearch, deduce_matches, find_specific_pattern, find_repeat, find_cycle, UnionFind, Octree, LazySegmentTree, Trie
from aocutils.visuals import visualize_graph, labelize, animate_grid, plot
from aocutils.cfg import CFG
from aocutils.earley import State, Earley
from aocutils.shunting import ShuntingYard

In [46]:
# | export

def gridneigh(inp, to_dict = True, diag=False, inc_self=False, parser=None):
    """
    Example to parse a simple grid consisting of ints
    When parser is None, characters will be parsed
    --------
    >>> grid, neigh = gridneigh('input.txt', parser=lambda x: [int(ch) for ch in x.split(',')])
    """

    if not parser:
        arr = [list(line) for line in inp.split('\n')]
    else:
        arr = [parser(line) for line in inp.split('\n')]
    neigh = arr_neighbors(arr,diag, inc_self)
    if to_dict:
        arr = arr_to_dict(arr)
    return arr, neigh    

In [82]:
grid, ins = open("in.txt").read().split("\n\n")
grid = [list(g) for g in grid.split('\n')]

nodes = {}
neigh = {}

deltas = ()
top = {}
bottom = {}
left = {}
right = {}


for r, row in enumerate(grid,0):
    for c, val in enumerate(row,0):
        if val == '.' or val == '#':
            nodes[complex(c,r)] = val
rows = len(grid)
cols = len(grid[0])

In [83]:

qua = defaultdict(set)
for node in nodes:
    if node.imag < 50:
        if node.real < 100:
            qua[1].add(node)
        else:
            qua[2].add(node)
    elif node.imag < 100:
        qua[3].add(node)
    elif node.imag < 150:
        if node.real < 50:
            qua[4].add(node)
        else:
            qua[5].add(node)
    else:
        qua[6].add(node)
for k,v in qua.items():
    print(k, len(v))

1 2500
2 2500
3 2500
4 2500
5 2500
6 2500


In [84]:
discounts = {
    1 : (0,50),
    2 : (0,100),
    3 : (50,50),
    4 : (100,0),
    5 : (100,50),
    6 : (150,0),
}
def getdiscount(node, qua, trans):
    disim, disre = discounts[qua]
    if not trans:
        return complex(node.real - disre, node.imag - disim)
    else:
        return complex(node.real - disre, 49 - (node.imag - disim))
def getrevdiscount(node, qua):
    disim, disre = discounts[qua]
    return complex(node.real + disre, node.imag + disim)
        


In [85]:
discountedqua = defaultdict(set)
for k, v in qua.items():
    for node in v:
        discountedqua[k].add(getdiscount(node, k,0))

In [86]:
Q = namedtuple('Q', 'top bottom left right')
sides = {}
for k, v in discountedqua.items():
    top = {c: min([node for node in v if node.real == c], key = lambda x:x.imag, default = '99999') for c in range(0, cols)}
    top = {i:j for i,j in top.items() if j != '99999'}
    bottom = {c: min([node for node in v if node.real == c], key = lambda x:-x.imag, default = '99999') for c in range(0, cols)}
    bottom = {i:j for i,j in bottom.items() if j != '99999'}
    left = {r: min([node for node in v if node.imag == r], key = lambda x:x.real, default = '99999') for r in range(0, rows)}
    left = {i:j for i,j in left.items() if j != '99999'}
    right = {r: min([node for node in v if node.imag == r], key = lambda x:-x.real, default = '99999') for r in range(0, rows)}
    right = {i:j for i,j in right.items() if j != '99999'}
    sides[k] = Q(top, bottom, left, right)
    

In [87]:
top = {c: min([node for node in nodes if node.real == c], key = lambda x:x.imag) for c in range(cols)}
bottom = {c: min([node for node in nodes if node.real == c], key = lambda x:-x.imag) for c in range(cols)}
left = {r: min([node for node in nodes if node.imag == r], key = lambda x:x.real) for r in range(rows)}
right = {r: min([node for node in nodes if node.imag == r], key = lambda x:-x.real) for r in range(rows)}

In [98]:

        
        
directions = {'right' : complex(1,0),
              'left' : complex(-1,0),
              'up' : complex(0,-1),
              'down' : complex(0,+1)}
revdirections = rev(directions)

conn = {
    (1, 'right') :  (2, 'right', 0,0),
    (1, 'left'):    (4, 'right', 1,0),
    (1, 'up'):      (6, 'right', 0,1),
    (1, 'down'):    (3, 'down', 0,0),
    (2, 'right') :  (5, 'left', 1,0),
    (2, 'left'):    (1, 'left', 0,0),
    (2, 'up'):      (6, 'up', 0,0),
    (2, 'down'):    (3, 'left', 0,1),
    (3, 'right') :  (2, 'up', 0,1),
    (3, 'left'):    (4, 'down', 0,1),
    (3, 'up'):      (1, 'up', 0,0),
    (3, 'down'):    (5, 'down', 0,0),
    (4, 'right') :  (5, 'right', 0,0),
    (4, 'left'):    (1, 'right', 1,0),
    (4, 'up'):      (3, 'right', 0,1),
    (4, 'down'):    (6, 'down', 0,0),
    (5, 'right') :  (2, 'left', 1,0),
    (5, 'left'):    (4, 'left', 0,0),
    (5, 'up'):      (3, 'up', 0,0),
    (5, 'down'):    (6, 'left', 0,1),
    (6, 'right') :  (5, 'up', 0,1),
    (6, 'left'):    (1, 'down', 0,1),
    (6, 'up'):      (4, 'up', 0,0),
    (6, 'down'):    (2, 'down', 0,0),
}



In [102]:
move = ints(ins)
direc = []
for i in ins:
    if i == 'L':
        direc.append(complex(0,-1))
    if i == 'R':
        direc.append(complex(0,1))

# right = complex(1,0)
# left = complex(-1,0)

cur = 1
d = complex(1,0)
loc = left[0]
print('start', loc)
pnt = 0
while pnt < len(move):
    m = move[pnt]
    for _ in range(m):
        new = loc + d
        newdir = d
        newcur = cur

        #
        # if new not in nodes:
        #     if d == complex(1,0):
        #         new = left[loc.imag]
        #     if d == complex(-1,0):
        #         new = right[loc.imag]
        #     if d == complex(0,1):
        #         new = top[loc.real]
        #     if d == complex(0,-1):
        #         new = bottom[loc.real]     
        
        if new not in qua[cur]:
            # for example quadrant 2, go up and tranpose or not
            newcur, newdir, trans, switch = conn[cur, revdirections[d]]
            print('was', loc, revdirections[d], 'qua', cur,  'newqua', newcur, newdir, trans, switch)
            
            origdiscounted = getdiscount(loc, cur,trans)
            discounted = origdiscounted
            if switch:
                discounted = complex(origdiscounted.imag, origdiscounted.real)
            
            assert 0 <= discounted.real < 50
            assert 0 <= discounted.imag < 50
            
            if newdir == 'left':
                new = sides[newcur].right[discounted.imag]
            if newdir == 'right':
                new = sides[newcur].left[discounted.imag]
            if newdir == 'up':
                new = sides[newcur].bottom[discounted.real]
            if newdir == 'down':
                new = sides[newcur].top[discounted.real]
            newdir = directions[newdir]
            print('discounted', discounted, 'newdiscounted', new)
            new = getrevdiscount(new, newcur)
            print('newloc and dir', new, revdirections[d], cur)
            print('')
             
            
    
        if nodes[new] == '#':
            # print('wall')
            pass
        else:
            loc = new
            d = newdir
            cur = newcur
        
    
    if pnt < len(direc):
        # change direc
        d *= direc[pnt]
 
    
    else:
        pass
    pnt += 1
    
loc

# dont forget last movement

if d == complex(1,0):
    ans = 0
if d == complex(-1,0):
    ans = 2
if d == complex(0,1):
    ans = 1
if d == complex(0,-1):
    ans = 3
ans += (loc.imag+1)*1000+ (loc.real+1)*4
ans



start (50+0j)
was (50+0j) left qua 1 newqua 4 right 1 0
discounted 49j newdiscounted 49j
newloc and dir 149j left 1

was 142j left qua 4 newqua 1 right 1 0
discounted 7j newdiscounted 7j
newloc and dir (50+7j) left 4

was (50+29j) left qua 1 newqua 4 right 1 0
discounted 20j newdiscounted 20j
newloc and dir 120j left 1

was 131j left qua 4 newqua 1 right 1 0
discounted 18j newdiscounted 18j
newloc and dir (50+18j) left 4

was (50+14j) left qua 1 newqua 4 right 1 0
discounted 35j newdiscounted 35j
newloc and dir 135j left 1

was (3+149j) down qua 4 newqua 6 down 0 0
discounted (3+49j) newdiscounted (3+0j)
newloc and dir (3+150j) down 4

was 162j left qua 6 newqua 1 down 0 1
discounted (12+0j) newdiscounted (12+0j)
newloc and dir (62+0j) left 6

was (50+37j) left qua 1 newqua 4 right 1 0
discounted 12j newdiscounted 12j
newloc and dir 112j left 1

was 126j left qua 4 newqua 1 right 1 0
discounted 23j newdiscounted 23j
newloc and dir (50+23j) left 4

was (50+37j) left qua 1 newqua 4 right

55267.0

In [ ]:
len(sides[2].left), len(sides[1].right), len(sides[5].bottom)

(50, 50, 50)

In [57]:
complex(50,26) in qua[1]

True

In [ ]:
28592 too low

In [72]:
move = ints(ins)
direc = []
for i in ins:
    if i == 'L':
        direc.append(complex(0,-1))
    if i == 'R':
        direc.append(complex(0,1))



d = complex(1,0)
loc = left[1]
pnt = 0
while pnt < len(move):
    m = move[pnt]
    for _ in range(m):
        new = loc + d
        if new not in nodes:
            if d == complex(1,0):
                new = left[loc.imag]
            if d == complex(-1,0):
                new = right[loc.imag]
            if d == complex(0,1):
                new = top[loc.real]
            if d == complex(0,-1):
                new = bottom[loc.real]
    
        if nodes[new] == '#':
            pass
        else:
            loc = new
            print(new)
        
    
    if pnt < len(direc):
        # change direc
        d *= direc[pnt]
        print(d)
    
    else:
        pass
    pnt += 1
    
loc

# dont forget last movement

if d == complex(1,0):
    ans = 0
if d == complex(-1,0):
    ans = 2
if d == complex(0,1):
    ans = 1
if d == complex(0,-1):
    ans = 3
ans += loc.imag*1000+ loc.real*4
ans



(51+1j)
(52+1j)
(53+1j)
(54+1j)
(55+1j)
(56+1j)
1j
(56+2j)
(56+3j)
(56+4j)
(56+5j)
(56+6j)
(56+7j)
(56+8j)
(56+9j)
(56+10j)
(56+11j)
(56+12j)
(1+0j)
(57+12j)
(58+12j)
1j
(-1+0j)
(57+12j)
(56+12j)
(55+12j)
1j
(55+13j)
(55+14j)
(55+15j)
(55+16j)
(55+17j)
(55+18j)
(55+19j)
(55+20j)
(55+21j)
(55+22j)
(1+0j)
(56+22j)
1j
(56+23j)
(56+24j)
(56+25j)
(56+26j)
(-1+0j)
(55+26j)
(54+26j)
(53+26j)
(52+26j)
(51+26j)
(50+26j)
(149+26j)
(148+26j)
(147+26j)
(146+26j)
(145+26j)
(144+26j)
(143+26j)
(142+26j)
(141+26j)
(140+26j)
1j
(140+27j)
(1+0j)
(141+27j)
(142+27j)
(143+27j)
(144+27j)
(145+27j)
(146+27j)
(147+27j)
1j
(147+28j)
(147+29j)
(147+30j)
(147+31j)
(147+32j)
(147+33j)
(147+34j)
(147+35j)
(147+36j)
(147+37j)
(147+38j)
(147+39j)
(147+40j)
(147+41j)
(147+42j)
(147+43j)
(147+44j)
(147+45j)
(147+46j)
(147+47j)
(147+48j)
(147+49j)
(147+0j)
(147+1j)
(147+2j)
(147+3j)
(147+4j)
(147+5j)
(147+6j)
(147+7j)
(147+8j)
(147+9j)
(147+10j)
(147+11j)
(-1+0j)
(146+11j)
(145+11j)
(144+11j)
(143+11j)
(142+11j)
(141

161182.0